In [1]:
# import pyfunctions from support functions
import sys
sys.path.insert(0, '../00_support_functions/') 
from pyfunctions import *

from moco_pyfunctions import *

In [ ]:
# cell to evaluate ANTs rigid, affine, and deformation algorithms

import os, re
import time

# Replace '/path/to/your/directory' with the actual root directory you want to search
root_directory = '../evaldata'
match_string = 'dce.nii'  # Replace with the string you're looking for

files = find_files_matching_string(root_directory, match_string)

n = len(files)
c = 0
ssim_results = np.zeros((n, 3))
time_taken = np.zeros((n, 3))
for file in files:

    # we split input filenames so we can easily name the output filenames 
    dirname, basename = os.path.split(file)

    split_filename = file.split("/")
    subj = split_filename[2]
    sess = split_filename[3]

    output_name_1 = re.sub("dce", "moco_ants_rigid", basename)
    output_name_2 = re.sub("dce", "moco_ants_affine", basename)
    output_name_3 = re.sub("dce", "moco_ants_defrom", basename)
    outdir = r'evalresults'
    output_1 = os.path.join(outdir, output_name_1)
    output_2 = os.path.join(outdir, output_name_2)
    output_3 = os.path.join(outdir, output_name_3)

    start = time.time()
    moco_ants(file, output_1, "rigid")
    end = time.time()
    time_taken[c, 0] = end-start

    start = time.time()
    moco_ants(file, output_2, "affine")
    end = time.time()
    time_taken[c, 1] = end-start
    
    start = time.time()
    moco_ants(file, output_3, "deform")
    end = time.time()
    time_taken[c, 2] = end-start

    ssim_results[c, 0] = eval_moco_ssim(file, output_1)
    ssim_results[c, 1] = eval_moco_ssim(file, output_2)
    ssim_results[c, 2] = eval_moco_ssim(file, output_3)

    c += 1


import pandas as pd
cn = ["ants (rigid)", "ants (affine)", "ants (deform)"]
ssim_mc_deform = pd.DataFrame(ssim_results, columns=cn)
runtimes_mc_deform = pd.DataFrame(time_taken, columns=cn)

ssim_mc_deform.to_csv(r'evalresults/ssim_mc_ants.csv')
runtimes_mc_deform.to_csv(r'evalresults/runtime_mc_ants.csv')

In [ ]:
# cell to calculate moco using niftyreg deformation algorithm using default, 5x
# and 10x regularization params,

import os, re
import time

# Replace '/path/to/your/directory' with the actual root directory you want to search
root_directory = '../evaldata'
match_string = 'dce.nii'  # Replace with the string you're looking for

files = find_files_matching_string(root_directory, match_string)

n = len(files)
c = 0
ssim_results = np.zeros((n, 3))
time_taken = np.zeros((n, 3))
for file in files:

    # we split input filenames so we can easily name the output filenames 
    dirname, basename = os.path.split(file)

    split_filename = file.split("/")
    subj = split_filename[2]
    sess = split_filename[3]

    output_name_1 = re.sub("dce", "moco_niftyreg_05x", basename)
    output_name_2 = re.sub("dce", "moco_niftyreg_def", basename)
    output_name_3 = re.sub("dce", "moco_niftyreg_10x", basename)
    outdir = r'evalresults'
    output_1 = os.path.join(outdir, output_name_1)
    output_2 = os.path.join(outdir, output_name_2)
    output_3 = os.path.join(outdir, output_name_3)

    start = time.time()
    moco_nrdef(file, output_1)
    end = time.time()
    time_taken[c, 0] = end-start

    start = time.time()
    moco_nrdef(file, output_2, be='0.001', le='0.01')
    end = time.time()
    time_taken[c, 1] = end-start
    
    start = time.time()
    moco_nrdef(file, output_3, be='0.01', le='0.1')
    end = time.time()
    time_taken[c, 2] = end-start

    ssim_results[c, 0] = eval_moco_ssim(file, output_1)
    ssim_results[c, 1] = eval_moco_ssim(file, output_2)
    ssim_results[c, 2] = eval_moco_ssim(file, output_3)

    c += 1


import pandas as pd
cn = ["niftyreg (5x)", "niftyreg (default)", "niftyreg (10x)"]
ssim_mc_deform = pd.DataFrame(ssim_results, columns=cn)
runtimes_mc_deform = pd.DataFrame(time_taken, columns=cn)

ssim_mc_deform.to_csv(r'evalresults/ssim_mc_niftyreg.csv')
runtimes_mc_deform.to_csv(r'evalresults/runtime_mc_niftyreg.csv')

In [ ]:
# cell to try out ITKs elastix deformation algorithm

import os, re
import time

# Replace '/path/to/your/directory' with the actual root directory you want to search
root_directory = '../evaldata'
match_string = 'dce.nii'  # Replace with the string you're looking for

files = find_files_matching_string(root_directory, match_string)

n = len(files)
c = 0
ssim_results = np.zeros(n)
time_taken = np.zeros(n)
for file in files:

    # we split input filenames so we can easily name the output filenames 
    dirname, basename = os.path.split(file)

    split_filename = file.split("/")
    subj = split_filename[2]
    sess = split_filename[3]

    output_name_1 = re.sub("dce", "moco_elastix", basename)
    outdir = r'evalresults'
    output_1 = os.path.join(outdir, output_name_1)

    start = time.time()
    moco_elastix(file, output_1)
    end = time.time()
    time_taken[c] = end-start

    ssim_results[c] = eval_moco_ssim(file, output_1)

    c += 1


import pandas as pd
cn = ["elastix"]
ssim_mc_deform = pd.DataFrame(ssim_results, columns=cn)
runtimes_mc_deform = pd.DataFrame(time_taken, columns=cn)

ssim_mc_deform.to_csv(r'evalresults/ssim_mc_elastix.csv')
runtimes_mc_deform.to_csv(r'evalresults/runtime_mc_elastix.csv')